In [26]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

from sklearn import datasets
from sklearn.datasets.mldata import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tempfile


import time
import numpy as np
import pandas as pd
import re

In [8]:
url ="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

'''
age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

'''


'\nage: continuous.\nworkclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.\nfnlwgt: continuous.\neducation: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.\neducation-num: continuous.\nmarital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.\noccupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.\nrelationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.\nrace: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.\nsex: Female, Male.\ncapital-gain: continuous.\ncapital-loss: continuous.\nhours-per-week: continuous.\nnative-country: United-States, Cambodia, En

In [45]:
def clean_data( data):
    
        #data.fillna(0. , inplace= True)
        data = data.astype(str)
        
        for col in data.columns:
            data[col]=data[col].map(str.strip) # removing all space from the string values

       
    # age: continuous.
        workclass_map={ 'Private':1, 'Self-emp-not-inc':2, 'Self-emp-inc':3, 'Federal-gov':4, 'Local-gov':5
                   , 'State-gov':6, 'Without-pay':7, 'Never-worked':8 }
    # fnlwgt: continuous.
        education_map ={'Bachelors':1, 'Some-college':2, '11th':3, 'HS-grad':4, 'Prof-school':5, 'Assoc-acdm':6
                    , 'Assoc-voc':7, '9th':8, '7th-8th':9, '12th':10, 'Masters':11, '1st-4th':12
                    , '10th':13, 'Doctorate':14, '5th-6th':15, 'Preschool':16 }
    # education-num: continuous.
        marital_status_map={ 'Married-civ-spouse':1, 'Divorced':2, 'Never-married':3, 'Separated':4
                        , 'Widowed':4, 'Married-spouse-absent':6, 'Married-AF-spouse':7}
        occupation_map={ 'Tech-support':1, 'Craft-repair':2, 'Other-service':3, 'Sales':4, 'Exec-managerial':5
                    , 'Prof-specialty':6, 'Handlers-cleaners':7, 'Machine-op-inspct':8, 'Adm-clerical':9
                    , 'Farming-fishing':10, 'Transport-moving':11, 'Priv-house-serv':12
                    , 'Protective-serv':13, 'Armed-Forces':14}
        relationship_map={ 'Wife':1, 'Own-child':2, 'Husband':3, 'Not-in-family':4, 'Other-relative':5, \
                          'Unmarried':6}
        race_map ={'White':1, 'Asian-Pac-Islander':2, 'Amer-Indian-Eskimo':3, 'Other':4, 'Black':5}
        sex_map={'Female':1, 'Male':2}
        # capital-gain: continuous.
        # capital-loss: continuous.
        # hours-per-week: continuous.
        native_country_map ={'United-States':1, 'Cambodia':2, 'England':3, 'Puerto-Rico':4, 'Canada':5
                             , 'Germany':6, 'Outlying-US(Guam-USVI-etc)':7, 'India':8, 'Japan':9, 'Greece':10
                             , 'South':11, 'China':12, 'Cuba':13, 'Iran':14, 'Honduras':15, 'Philippines':16
                             , 'Italy':17, 'Poland':18, 'Jamaica':19, 'Vietnam':20, 'Mexico':21
                             , 'Portugal':21, 'Ireland':22
                             , 'France':23, 'Dominican-Republic':24, 'Laos':25, 'Ecuador':26
                             , 'Taiwan':27, 'Haiti':28, 'Columbia':29
                             , 'Hungary':30, 'Guatemala':31, 'Nicaragua':32, 'Scotland':33
                             , 'Thailand':34, 'Yugoslavia':35
                             , 'El-Salvador':36, 'Trinadad&Tobago':37, 'Peru':38, 'Hong':39, \
                             'Holand-Netherlands':40}

        label_map={'<=50K':0, '>50K':1}
        
        theregex = re.compile(r'[^\d.-]+')

        data.replace(to_replace='?', value ='0') 

        data.replace(inplace =True ,to_replace = {'workclass':workclass_map 
        ,'education':education_map
                         ,'marital-status':marital_status_map 
                         ,'occupation':occupation_map , 'relationship':relationship_map
                          ,'race':race_map , 'sex':sex_map ,'native-country':native_country_map
                            ,'label': label_map
                         } )

        data = data.astype('str').applymap(lambda x: re.sub(r'[^\d.]+', '0', x))  

        data = data.astype('int')

        return pd.DataFrame(data)


def load_dataset( dataset_name):
        
              
        url ="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
        
        columns=['age' , 'workclass' ,'fnlwgt','education' ,'education-num','marital-status' ,'occupation', \
                 'relationship'
            ,'race' , 'sex' ,'capital-gain' ,'capital-loss' ,'hours-per-week' ,'native-country' ,'label']

        all_data = pd.read_csv(url ,  names=columns, index_col=False)
        
        all_data.reset_index()
        print(all_data.shape)
        
        
        cleaned_data= all_data
        cleaned_data = clean_data(all_data)
        
        return cleaned_data
        
        
 

In [46]:
df= load_dataset("")

(32561, 15)


In [56]:
#df

In [51]:
y= np.array(df['label'], dtype= np.int)
               
X = np.array(df.drop('label', axis =1), dtype= np.float)
print(y.shape , X.shape)

(32561,) (32561, 14)


In [52]:
X[:5]

array([[  3.90000000e+01,   6.00000000e+00,   7.75160000e+04,
          1.00000000e+00,   1.30000000e+01,   3.00000000e+00,
          9.00000000e+00,   4.00000000e+00,   1.00000000e+00,
          2.00000000e+00,   2.17400000e+03,   0.00000000e+00,
          4.00000000e+01,   1.00000000e+00],
       [  5.00000000e+01,   2.00000000e+00,   8.33110000e+04,
          1.00000000e+00,   1.30000000e+01,   1.00000000e+00,
          5.00000000e+00,   3.00000000e+00,   1.00000000e+00,
          2.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.30000000e+01,   1.00000000e+00],
       [  3.80000000e+01,   1.00000000e+00,   2.15646000e+05,
          4.00000000e+00,   9.00000000e+00,   2.00000000e+00,
          7.00000000e+00,   4.00000000e+00,   1.00000000e+00,
          2.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.00000000e+01,   1.00000000e+00],
       [  5.30000000e+01,   1.00000000e+00,   2.34721000e+05,
          3.00000000e+00,   7.00000000e+00,   1.00000000e+0